<a href="https://colab.research.google.com/github/keshsri/machine-learning/blob/main/IMDB_Reviews_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [ ]:
training_sentences = []
training_labels =[]

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [ ]:
import tensorflow as tf

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_2 (Dense)             (None, 6)                 102       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160109 (625.43 KB)
Trainable params: 160109 (625.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_2"
__________________________________________________________

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model1.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
num_epochs = 10

model.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

model1.fit(padded,
          training_labels_final,
          epochs=num_epochs,
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 9s 10ms/step - loss: 0.5648 - accuracy: 0.7341 - val_loss: 0.3981 - val_accuracy: 0.8421
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3361 - accuracy: 0.8633 - val_loss: 0.3325 - val_accuracy: 0.8584
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2690 - accuracy: 0.8903 - val_loss: 0.3311 - val_accuracy: 0.8585
Epoch 4/10
782/782 [==============================] - 10s 12ms/step - loss: 0.2327 - accuracy: 0.9103 - val_loss: 0.3461 - val_accuracy: 0.8542
Epoch 5/10
782/782 [==============================] - 9s 12ms/step - loss: 0.2062 - accuracy: 0.9226 - val_loss: 0.3709 - val_accuracy: 0.8476
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.1844 - accuracy: 0.9339 - val_loss: 0.3964 - val_accuracy: 0.8412
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.1673 - accuracy: 0.9400 - val_loss: 0.4284 - val_accuracy: 0.8367

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
import io

reverse_word_index = tokenizer.index_word

out_v = io.open('vecs.tsv', 'w', encoding='utf8')
out_m = io.open('meta.tsv', 'w', encoding='utf8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()